In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [ ]:
class DenseLayer:
    def __init__(self, input_size, output_size, activatoin=None):
        self.weights = tf.Variable(
            tf.random.normal([input_size, output_size]))
        self.bias = tf.Variable(
            tf.random.normal([1, output_size]))
        self.activation = activatoin
        self.m_w, self.v_w = tf.Variable(0.0), tf.Variable(0.0)
        self.m_b, self.v_b = tf.Variable(0.0), tf.Variable(0.0)


    def sigmoid(self, data):
        return tf.divide(1, (tf.add(1, np.exp(-data))))

    def sigmoid_der(self, data):
        return tf.multiply(self.sigmoid(data), (1-self.sigmoid(data)))

    def relu(self, data):
        data[np.where(data <= 0)] = 0

    def relu_der(self, data):
        return np.where(data > 0 , 1 , 0)

    def adam(self, grd_w, grd_b, t, eta=0.01, betta1=0.9, betta2=0.999, eps=1e-8):
        self.m_w = tf.add(tf.multiply(self.m_w,betta1), tf.multiply(1-betta1, grd_w))
        self.m_b = tf.add(tf.multiply(self.m_b, betta1), tf.multiply(1-betta1, grd_b))
        self.v_w = tf.add(tf.multiply(self.v_w, betta2), tf.multiply(1-betta2, tf.multiply(grd_w, grd_w)))
        self.v_b = tf.add(tf.multiply(self.v_b, betta2, tf.multiply(1-betta2, tf.multiply(grd_b, grd_b))))

        new_m_w = tf.divide(self.m_w, 1-tf.square(betta1, t))
        new_m_b = tf.divide(self.m_b, 1-tf.square(betta1, t))
        new_v_w = tf.divide(self.v_w, 1-tf.square(betta2, t))
        new_v_b = tf.divide(self.v_b, 1-tf.square(betta2, t))

        self.weights -= tf.multiply(eta, tf.divide(new_m_w, tf.sqrt(new_v_w+eps)))
        self.bias -= tf.multiply(eta, tf.divide(new_m_b, tf.sqrt(new_v_b+eps)))

    def forward(self, input_data):
        self.inputs = input_data

        self.output = tf.matmul(self.inputs, self.weights) + self.bias

        if self.activation == 'sigmoid':
            self.output = self.sigmoid(self.output)
        if self.activation == 'relu':
            self.output = self.relu(self.output)

        return self.output

    def backward(self, error, lr, i):
        if self.activation == 'sigmoid':
            error = self.sigmoid_der(error)

        if self.activation == 'relu':
            error = self.relu_der(error)

        grad_weights = tf.multiply(self.inputs, error)
        grad_input = tf.multiply(error, tf.transpose(self.weights))

        self.adam(grad_weights, error.sum(axis=0), i)
        return grad_input

In [ ]:
class DenseNetwork:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)

    def loss(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_true-y_pred))

    def call(self, X_train, y_train, lr, epoch):
        n_sample = X_train.shape[0]
        for i in range(1, epoch+1):
            input = tf.Variable(X_train, dtype=tf.float64)
            err = 0
            for layer in self.layers:
                input = layer.forward(input)

            err += self.loss(y_train, input)
            error = 2*(input.flatten() - y_train) / len(y_train)
            for layer in reversed(self.layers):
                error = layer.backward(error, lr, i)

            err /= n_sample
            print(f"epoch {i}, Loss {err}")